In [1]:
import os
os.chdir("/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/Data/Visium/raw/T-GRAASP/")
import torch
from models import STx_ARGVA, STx_encoder, STx_discriminator
from utils import *
from train import Trainer

import warnings
from scipy.stats import ConstantInputWarning
warnings.filterwarnings("ignore", category=ConstantInputWarning)

# sample_ids = ['CAQSC','WYYBT','XXGBT','ZWFBT','ZWJBT']
sample_ids = ['Young1','Mid1','Old1','Young2', 'Mid2',  'Old2']
# sample_ids = ['Young1']
raw_dir = '/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/Data/Visium/raw'
test_ratio = 0.1


sp_graph_list, sp_adj_list = SGEDataset(sample_ids, raw_dir, test_ratio=0.10)
# sc_graph_list = SGEDataset1(sample_ids, raw_dir)

N_nodes   = sp_graph_list[0].x.size(0)   # 节点个数
F_inputs  = sp_graph_list[0].x.size(1)   

ppi_file     = f'{raw_dir}/PPI1.connect.txt'
connectivity = load_ppi_connectivity(ppi_file, F_inputs)    # shape = [2, E]




Loading sample Young1 
Loading sample Mid1 
Loading sample Old1 
Loading sample Young2 
Loading sample Mid2 
Loading sample Old2 


In [2]:
encoder = STx_encoder(
    in_channels      = F_inputs,        
    hidden_channels  = 96,
    out_channels     = 16,
    m = 29, l = 3, K = 3,
    connect = connectivity,        
    pi = 0.75,
    n_heads = 8,
    activate_sc_alignment=False
)

discriminator = STx_discriminator(
    in_channels     = 16,  
    hidden_channels = 8,   
    out_channels    = 1
)

model = STx_ARGVA(
    encoder      = encoder,
    discriminator= discriminator,
    l            = 3          
)

In [3]:
model.load_state_dict(torch.load('/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/Data/Visium/raw/T-GRAASP/checkpoints_multi/best_model_auc_mean.pkl'))

trainer = Trainer(
    device="cuda:1",
    model=model,
    F_inputs = F_inputs,
    sp_graph_list=sp_graph_list[0:3],
    sc_graph_list=sp_graph_list[3:6],
    sp_adj_list=sp_adj_list[0:3]
    )

best_model_state = trainer.finetune(patience=1000, lr=1e-3, epochs=5)
torch.save(best_model_state,'/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/Data/Visium/raw/T-GRAASP/checkpoints_multi/finetuned_best_model.pkl')


 Start Fine-tuning Alignment between spatial and scRNA
Creating new alignment model...
[Finetune] Epoch 001 | Avg Loss: 79.884384 | Avg Align Loss (Train): 79.884384 | Avg Align Loss (Eval): 83.750951 | Avg Corr: nan | Avg AUC: 0.6330
